# UCI ML Repository: Heart Disease Data Set
---
### Case Optimum - Paulo Henrique Spada de Moura (Julho/2020)
### Aplicação de modelo Random Forest para predição de doenças cardíacas

* Importação das bibliotecas básicas (pandas, numpy, etc) + algoritmo de Random Forest do sklearn:

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import roc_curve, auc 
from sklearn.metrics import classification_report 
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split

import eli5 #realizei o pip install para o pacote -> 'pip install eli5'
from eli5.sklearn import PermutationImportance

np.random.seed(123)

import warnings
warnings.filterwarnings('ignore')

* Importação do data set através do repositório da UCI e inserção dos nomes das colunas para identificação dos atributos

In [ ]:
colunas = ['age','sex','cp','trstbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','num-target']
file = 'https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data'
heart = pd.read_csv(file, names=colunas)

In [ ]:
heart.head()

In [ ]:
heart.info()

De acordo com a descrição do data set, devemos realizar alguns ajustes nas variáveis categóricas para representação mais acurada das informações ao modelo que utilizaremos. Algumas das categorias representadas por números receberão as respectivas labels para melhor aplicação do modelo e interpretação dos resultados:

In [ ]:
heart['sex'][heart['sex'] == 0] = 'female'
heart['sex'][heart['sex'] == 1] = 'male'

heart['cp'][heart['cp'] == 1] = 'typical angina'
heart['cp'][heart['cp'] == 2] = 'atypical angina'
heart['cp'][heart['cp'] == 3] = 'non-anginal pain'
heart['cp'][heart['cp'] == 4] = 'asymptomatic'

heart['fbs'][heart['fbs'] == 0] = '< 120mg/dl'
heart['fbs'][heart['fbs'] == 1] = '> 120mg/dl'

heart['restecg'][heart['restecg'] == 0] = 'normal'
heart['restecg'][heart['restecg'] == 1] = 'ST-T abnormality'
heart['restecg'][heart['restecg'] == 2] = 'LVH'

heart['exang'][heart['exang'] == 0] = 'no'
heart['exang'][heart['exang'] == 1] = 'yes'

heart['slope'][heart['slope'] == 1] = 'upsloping'
heart['slope'][heart['slope'] == 2] = 'flat'
heart['slope'][heart['slope'] == 3] = 'downsloping'

heart['thal'][heart['thal'] == '3.0'] = 'normal'
heart['thal'][heart['thal'] == '6.0'] = 'fixed defect'
heart['thal'][heart['thal'] == '7.0'] = 'reversable defect'

In [ ]:
heart.head()

Novamente, de acordo com a descrição da base, em relação à nossa coluna target (presença ou não de doença cardíaca), os valores **1,2,3 e 4** indicam a **presença** de doença cardíaca, enquanto o valor **0 (zero)** indica que não há presença de doença no coração. Dessa forma, utilizaremos a transformação dos valores na coluna target:

In [ ]:
heart['num-target'][heart['num-target'] == 1] = 1
heart['num-target'][heart['num-target'] == 2] = 1
heart['num-target'][heart['num-target'] == 3] = 1
heart['num-target'][heart['num-target'] == 4] = 1

In [ ]:
heart['num-target'].value_counts()

Além dos ajustes realizados acima, é necessária a transformação no tipo das variáveis para que sejam aplicadas corretamente no modelo (categóricas e numéricas):

In [ ]:
heart['age'] = heart['age'].astype('int64')
heart['sex'] = heart['sex'].astype('object')
heart['cp'] = heart['cp'].astype('object')
heart['fbs'] = heart['fbs'].astype('object')
heart['restecg'] = heart['restecg'].astype('object')
heart['exang'] = heart['exang'].astype('object')
heart['slope'] = heart['slope'].astype('object')
heart['num-target'] = heart['num-target'].astype('object')

In [ ]:
heart.info()

A função abaixo foi criada para apresentar os valores únicos por atributo em nosso data set. Com ela, podemos checar se existem valores "estranhos" à coluna e avaliarmos como substituí-los:

In [ ]:
def check_unicos():
        for col in colunas:
            print(heart[col].unique())
check_unicos()

Foarm identificados alguns caracteres estranhos como "?", que possivelmente representam valores não preenchidos no data set. Para realizar a limpeza e tratamento desses dados, substituiremos "?" por "NaN" (efetivamente nulos) e, em seguida, removeremos as linhas correspondentes à presença desses valores:

* Transformação em *NaN*:

In [ ]:
heart[heart == '?'] = np.nan
heart.isnull().sum()

* Remoção das linhas correspondentes à presença de *NaN*:

In [ ]:
heart = heart.dropna()
heart.isnull().sum()

In [ ]:
heart.info()

## Aplicação do Random Forest

Em termos de aplicação, por tratarmos de uma base de dados um pouco mais complexa, com dados não-linearmente separáveis, optaremos pelo algoritmo de Random Forest, possibilitando maior generalização do fit (em relação ao algoritmo de Árvore de Decisão, por exemplo) e balanceando parâmetros como precisão e overfitting. Em termos de aplicabilidade no mundo real, também observamos frequente utilização do modelo de Random Forest em contextos de medicina.

Para darmos início à aplicação do modelo, primeiramente **criaremos as dummies** para cada atributo categórico do data set:

In [ ]:
heart = pd.get_dummies(heart, drop_first=True)

In [ ]:
heart.head()

Iniciaremos a aplicação do modelo fazendo a divisão da base em 80%/20% para treino e teste, respectivamente, como sugestão das boas práticas (obs: após a criação das dummies, o novo nome da coluna target é **"num-target_1"**):

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(heart.drop('num-target_1',1), heart['num-target_1'], test_size = 0.2, random_state = 10)

In [ ]:
modelo = RandomForestClassifier(max_depth=5)
modelo.fit(X_train,y_train)

* Avaliação do modelo (análise da **matriz de confusão** para avaliação da acuracidade):

In [ ]:
y_predict = modelo.predict(X_test)
y_pred_quant = modelo.predict_proba(X_test)[:, 1]
y_pred_bin = modelo.predict(X_test)

In [ ]:
# Matriz de confusão:

confusion_matrix = confusion_matrix(y_test, y_pred_bin)
confusion_matrix

* Descrição dos parâmetros de sensibilidade e especificidade (VP = verdadeiro positivo; VN = verdadeiro negativo; FP = falso positivo; FN = falso negativo):
    > * **Sensibilidade**: VP/(VP+FN)
    > * **Especificidade**: VN/(VN+FP)

In [ ]:
total = sum(sum(confusion_matrix))

sensibilidade = confusion_matrix[0,0]/(confusion_matrix[0,0] + confusion_matrix[1,0])
print('Sensibilidade: ', sensibilidade)

especificidade = confusion_matrix[1,1]/(confusion_matrix[1,1] + confusion_matrix[0,1])
print('Especificidade: ', especificidade)

* Avaliação da **curva ROC** para a variável de classificação target binária:
    > A curva ROC é uma FDA (função de distribuição acumulada) para plotagem da taxa de "verdadeiros positivos" *versus* "falsos positivos", ou seja, da **sensibilidade** (ou **probabilidade de detecção**) *versus* a probabiliadde de "alarme falso";
    > No caso abaixo, temos "tfp = Taxa de Falsos Positivos" e "tvp = Taxa de Verdadeiros Positivos";

In [ ]:
tfp, tvp, thresholds = roc_curve(y_test, y_pred_quant)

fig, ax = plt.subplots()
ax.plot(tfp, tvp)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.rcParams['font.size'] = 10
plt.title('Curva ROC para classificação de doenças cardíacas')
plt.xlabel('Falsos positivos (1-especificidade)')
plt.ylabel('Verdadeiros positivos (sensibilidade)')
plt.grid(True)

Para avaliação da performance do modelo, podemos calcular a integral da função geradora da curva ROC ou, basicamente, o valor de **área abaixo da curva ROC ("AUC" - *Area Under the Curve*)**. Serão utilizados os parâmetros aproximados abaixo:
* **0.90 - 1.00** = excelente;
* **0.80 - 0.90** = bom;
* **0.70 - 0.80** = regular
* **0.60 - 0.70** = ruim
* **0.50 - 0.60** = falho

In [ ]:
auc(tfp,tvp)

Com base no resultado acima, podemos assumir uma acuracidade de, aproximadamente, **90,5%** para o modelo de Random Forest.

Apenas como complemento, podemos utilizar o recurso e permutação das variáveis (através do pacote **'elif5'**) para melhor compreensão das predições e os respectivos pesos das variáveis independentes no modelo:

In [ ]:
perm = PermutationImportance(modelo, random_state=1).fit(X_test, y_test)
eli5.show_weights(perm, feature_names = X_test.columns.tolist())

Observando a tabela acima, compreendemos que o fator de maior influência, dentre as variáveis consideradas no modelo, para ocorrência de doença cardíaca é a presença de uma desordem no sangue conhecida por *"Thalassemia"*, dentro de uma característica de "defeito reversível". De acordo com alguns dos artigos produzidos em cima do data set (UCI), a condição de defeito reversível (*"reversible defect"*) durante a prática de atividades físicas costuma indicar a existência de bloqueio nas artérias coronárias, levando à condição cardíaca.